In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hUJ3sZVg4xSHqm4uzpvd7l7Hd00zcVcjnuklMhLGkG4/edit#gid=2113528684')
sheet= sheet = wb.worksheet('comments')
data = sheet.get_all_values()

In [ ]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
df[['Comment']]

,Comment
1,"Tbh, if you put an iOS 14/15 style launcher sk..."
2,Nice weather widget on the thumbnail… nice
3,"if this was a folding iphone, people would be ..."
4,"let&#39;s be honest, it looks siiiiiiiiick 🔥 🔥 🔥"
5,"Thanks for the review, Awesome insight!!"
...,...
2662,First
2663,awa
2664,Bruh
2665,2


In [ ]:
import re
import string

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
# def remove_email(text):
  # email= re.compile(r'\S*@\S*\s?')
  # return email.sub(r'',text)

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


In [ ]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
df['Comment'].replace('', np.nan, inplace=True)
df.dropna(subset=['Comment'], inplace=True)

In [ ]:
#  !pip install wheel
#  !pip install ntlk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stopwords(line):
  word_tokens = word_tokenize(line)
  filtered_sentence = [w for w in word_tokens if not w in stop_words]
  return " ".join(filtered_sentence)

In [ ]:
df['Comment']=df['Comment'].apply(lambda x : remove_punct(x))
df['Comment']=df['Comment'].apply(lambda x: remove_emoji(x))
df['Comment']=df['Comment'].apply(lambda x : remove_URL(x))
df['Comment']=df['Comment'].apply(lambda x : remove_html(x))
df['Comment']=df['Comment'].apply(lambda x : remove_email(x))
df['Comment']=df['Comment'].apply(lambda x : remove_stopwords(x))
df['Comment']=df['Comment'].str.strip()
df['Comment']=df['Comment'].str.lower()
df[['Comment']]


,Comment
1,tbh put ios 1415 style launcher skin could con...
2,nice weather widget thumbnail… nice
3,folding iphone people would losing minds
4,let39s honest looks siiiiiiiiick
5,thanks review awesome insight
...,...
2662,first
2663,awa
2664,bruh
2665,2


In [ ]:
# Now a very simple way to remove repeating words from
# your dataset, following the idea that the shortest and
# longest ones are usually useless:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),     deacc=True))

In [ ]:
# scoring
# !pip install -U textblob
# !python -m textblob.download_corpora
from textblob import TextBlob
df['polarity'] = df['Comment'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.loc[df['polarity'] > 0, 'score'] = 1
df.loc[df['polarity'] <= 0, 'score'] = -1
df

,Name,Comment,Time,Likes,Reply Count,polarity,score
1,T-man A,tbh if you put an ios 1415 style launcher skin...,2021-08-27T13:38:28Z,0,0,-0.078125,-1.0
2,BumperTABBY21,nice weather widget on the thumbnail… nice,2021-08-27T13:38:27Z,0,0,0.600000,1.0
3,AuthenTech - Ben Schmanke,if this was a folding iphone people would be l...,2021-08-27T13:38:25Z,0,0,0.000000,-1.0
4,AuthenTech - Ben Schmanke,let39s be honest it looks siiiiiiiiick,2021-08-27T13:37:35Z,0,0,0.600000,1.0
5,Robert Prescott,thanks for the review awesome insight,2021-08-27T13:36:54Z,0,0,0.600000,1.0
...,...,...,...,...,...,...,...
2662,Noah Van Winden,first,2021-08-26T23:07:38Z,1,0,0.250000,1.0
2663,leowon,awa,2021-08-26T23:07:38Z,0,0,0.000000,-1.0
2664,MoochieMancityfours -,bruh,2021-08-26T23:07:38Z,2,0,0.000000,-1.0
2665,Umar Mehfooz Jalali,2,2021-08-26T23:07:38Z,0,0,0.000000,-1.0


In [ ]:
# to download and inspect the dataset
# filename = 'Dataset.csv'

# df.to_csv(filename,index=False)

# print('Saved file: ' + filename)

In [ ]:
df

,Name,Comment,Time,Likes,Reply Count,polarity,score
1,T-man A,tbh put ios 1415 style launcher skin could con...,2021-08-27T13:38:28Z,0,0,-0.078125,-1.0
2,BumperTABBY21,nice weather widget thumbnail… nice,2021-08-27T13:38:27Z,0,0,0.600000,1.0
3,AuthenTech - Ben Schmanke,folding iphone people would losing minds,2021-08-27T13:38:25Z,0,0,0.000000,-1.0
4,AuthenTech - Ben Schmanke,let39s honest looks siiiiiiiiick,2021-08-27T13:37:35Z,0,0,0.600000,1.0
5,Robert Prescott,thanks review awesome insight,2021-08-27T13:36:54Z,0,0,0.600000,1.0
...,...,...,...,...,...,...,...
2662,Noah Van Winden,first,2021-08-26T23:07:38Z,1,0,0.250000,1.0
2663,leowon,awa,2021-08-26T23:07:38Z,0,0,0.000000,-1.0
2664,MoochieMancityfours -,bruh,2021-08-26T23:07:38Z,2,0,0.000000,-1.0
2665,Umar Mehfooz Jalali,2,2021-08-26T23:07:38Z,0,0,0.000000,-1.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Comment'],
                                                  df['score'],
                                                  test_size =0.2,
                                                  random_state = 324)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [ ]:
tf_train.shape

(1421, 4952)

In [ ]:
print(vect.vocabulary_)

{'well': 4742, 'monica': 2831, 'still': 4138, 'owes': 3109, 'chandler': 757, '10': 0, 'bucks': 643, 'lumpy': 2617, 'yew': 4885, 'recurrently': 3600, 'man': 2667, 'discovery': 1237, 'experimentally': 1520, 'scratch': 3807, 'towards': 4445, 'versed': 4644, 'giraffe': 1828, 'null': 2990, 'imperfect': 2161, 'mascara': 2689, 'unaccountable': 4527, 'syrup': 4257, 'broadly': 627, 'trouble': 4478, 'credit': 1025, 'preauricularly': 3356, 'follow': 1674, 'homeless': 2073, 'employer': 1402, 'hilarious': 2043, 'educated': 1361, 'dessert': 1182, 'rachel': 3529, 'rock': 3702, 'sombrero': 4020, 'last': 2460, 'part': 3147, 'video': 4652, 'taken': 4270, 'movie': 2854, 'making': 2661, 'friends': 1737, 'reunion': 3669, 'know': 2419, 'site': 3958, 'download': 1292, 'episodes': 1442, 'subtitles': 4192, 'new': 2937, 'virgo': 4662, 'logistically': 2575, 'brush': 640, 'creek': 1026, 'atypically': 364, 'inject': 2208, 'frequent': 1728, 'grip': 1907, 'woebegone': 4802, 'bright': 622, 'shoulder': 3919, 'whats': 

# **LINEAR REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(tf_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr.score(tf_train,y_train)

0.9802955665024631

In [ ]:
lr.score(tf_test, y_test)

0.7612359550561798

In [ ]:
expected = y_test
predicted = lr.predict(tf_test)

In [ ]:
# import sklearn.metrics as metrics
# from sklearn.metrics import plot_confusion_matrix
# cf = metrics.confusion_matrix(expected,predicted, labels = [1,-1])
# print(cf)
# fig, ax= plot_confusion_matrix(comf_mat = cf, class_names = [1,-1])
# plt.show()

# **BERT**

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0
!pip install --upgrade pip
!pip install tensorflow==2.0.0-rc1
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn
!pip3 install sentencepiece
!pip3 install tf_sentencepiece

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization


In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        text = tokenizer.tokenize(text)

        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]

    if Dropout_num == 0:
        # Without Dropout
        out = Dense(1, activation='sigmoid')(clf_output)
    else:
        # With Dropout(Dropout_num), Dropout_num > 0
        x = Dropout(Dropout_num)(clf_output)
        out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-gpu

In [ ]:


from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

import torch

import warnings
warnings.simplefilter('ignore')

In [ ]:
# !pip install --upgrade tensorflow-gpu==2.2.0 --user
# !pip uninstall tensorflow-datasets
!pip install tensorflow-datasets==4.0.0
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import keras
from keras import backend as K
from keras import activations, initializers, regularizers, constraints, metrics
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import (Dense, Dropout, Activation, Flatten, Reshape, Layer,
                          BatchNormalization, LocallyConnected2D,
                          ZeroPadding2D, Conv2D, MaxPooling2D, Conv2DTranspose,
                          GaussianNoise, UpSampling2D, Input)
# from keras.utils import conv_utils, multi_gpu_model
from keras.layers import Lambda
# from keras.engine import Layer, InputSpec
# from keras.legacy import interfaces
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
# X_train, X_test, y_train, y_test
# tf_train = vect.fit_transform(X_train)
# tf_test = vect.transform(X_test)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(X_train.Comment.values, tokenizer, max_len=160)
test_input = bert_encode(X_test.Comments.values, tokenizer, max_len=160)
train_labels = X_train.score.values

AttributeError: ignored

# **GloVe**

In [ ]:
def create_corpus_new(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet)]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus_new(df)

KeyError: ignored

In [ ]:
# https://www.kaggle.com/vbmokin/nlp-eda-bag-of-words-tf-idf-glove-bert
# https://www.analyticssteps.com/blogs/sentiment-analysis-youtube-comments